In [20]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [21]:
data = pd.read_csv("C:/Users/Sudik/Downloads/diabetes (1).csv")
data

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,63,negative
764,2,122,70,27,0,36.8,27,negative
765,5,121,72,23,112,26.2,30,negative
766,1,126,60,0,0,30.1,47,positive


In [22]:
x = data.iloc[:, :-1].values
y_string = data.iloc[:, -1].values

In [23]:
y_int = []
for s in y_string:
    if (s == 'positive'):
        y_int.append(1)
    else:
        y_int.append(0)

In [24]:
y = np.array(y_int, dtype='float64')

In [25]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [26]:
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)

In [27]:
class Dataset(Dataset):
    
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [28]:
dataset = Dataset(x, y)

In [29]:
#load data
train_loader = torch.utils.data.DataLoader(dataset = dataset, batch_size = 32, shuffle = True)

In [30]:
print('There are {} batches in the dataset'.format(len(train_loader)))
for (x,y) in train_loader:
    print('For one interation (batch), there is:')
    print('Data: {}'.format(x.shape))
    print('Labels: {}'.format(y.shape))
    break

There are 24 batches in the dataset
For one interation (batch), there is:
Data: torch.Size([32, 7])
Labels: torch.Size([32, 1])


In [31]:
#building NN
class Model(nn.Module):
    
    def __init__(self, input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [32]:
net = Model(7, 1)
criteria = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9)

C:\Users\Sudik\anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [34]:
#training of nn
epochs = 200
for epoch in range(200):
    for (inputs, labels) in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        #forward prop
        outputs = net(inputs)
        #loss calc
        loss = criteria(outputs, labels)
        #clear gradient buffer -> to not accumulate buffer (w <-- w - lr*gradient)
        optimizer.zero_grad()
        #calc gradient (backprop)
        loss.backward() #matrix of gradient
        #update weights
        optimizer.step()
    #accuracy calc
    outputs = (outputs>0.5).float()
    accuracy = (outputs == labels).float().mean()
    #print stats
    print('Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}'.format(epoch+1, epochs, loss, accuracy))

Epoch 1/200, Loss: 0.809, Accuracy: 0.500
Epoch 2/200, Loss: 0.621, Accuracy: 0.656
Epoch 3/200, Loss: 0.493, Accuracy: 0.781
Epoch 4/200, Loss: 0.461, Accuracy: 0.750
Epoch 5/200, Loss: 0.493, Accuracy: 0.656
Epoch 6/200, Loss: 0.380, Accuracy: 0.844
Epoch 7/200, Loss: 0.444, Accuracy: 0.781
Epoch 8/200, Loss: 0.563, Accuracy: 0.688
Epoch 9/200, Loss: 0.506, Accuracy: 0.750
Epoch 10/200, Loss: 0.420, Accuracy: 0.781
Epoch 11/200, Loss: 0.397, Accuracy: 0.812
Epoch 12/200, Loss: 0.550, Accuracy: 0.781
Epoch 13/200, Loss: 0.378, Accuracy: 0.844
Epoch 14/200, Loss: 0.468, Accuracy: 0.844
Epoch 15/200, Loss: 0.387, Accuracy: 0.812
Epoch 16/200, Loss: 0.320, Accuracy: 0.906
Epoch 17/200, Loss: 0.311, Accuracy: 0.875
Epoch 18/200, Loss: 0.556, Accuracy: 0.719
Epoch 19/200, Loss: 0.434, Accuracy: 0.812
Epoch 20/200, Loss: 0.648, Accuracy: 0.688
Epoch 21/200, Loss: 0.518, Accuracy: 0.719
Epoch 22/200, Loss: 0.528, Accuracy: 0.781
Epoch 23/200, Loss: 0.404, Accuracy: 0.812
Epoch 24/200, Loss: 

Epoch 192/200, Loss: 0.479, Accuracy: 0.781
Epoch 193/200, Loss: 0.501, Accuracy: 0.719
Epoch 194/200, Loss: 0.374, Accuracy: 0.812
Epoch 195/200, Loss: 0.455, Accuracy: 0.719
Epoch 196/200, Loss: 0.330, Accuracy: 0.781
Epoch 197/200, Loss: 0.500, Accuracy: 0.812
Epoch 198/200, Loss: 0.350, Accuracy: 0.875
Epoch 199/200, Loss: 0.266, Accuracy: 0.812
Epoch 200/200, Loss: 0.479, Accuracy: 0.719
